In [1]:
import joblib
EXP_PATH_NAME="WACA-IF"
joblib.cpu_count()

32

In [2]:
# !pip install --upgrade pip

import os
from joblib import Parallel, delayed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dataclasses
from sklearn.svm import OneClassSVM
from dataclasses import asdict
from tqdm import tqdm
import warnings
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold # Feature selector

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./Classification_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb
RIVAL_CLASSIFIER_TYPE_LST=["OCSVM", "kNN", "LOF"]


np.random.seed(SEED)
print(f"Numpy Seed was set to: {SEED}")

print("Setup Complete")

SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neural Networks utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER

In [3]:
import sklearn
sklearn.__dir__()
sklearn.__version__

'1.0.2'

In [4]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
        
    user_ids: list
    num_sample_points_per_exp: int
    exp_begin_cutoff_idx: int
    exp_end_cutoff_idx: int
        
    
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    window_size: int  # After resampling
    ocsvm_step_width: int  # After resampling
    scaler: str  # StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_kernel: str # ocsvm kernel
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects=29,
    max_test_subjects=10,
    user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49],
    num_sample_points_per_exp=21000,
    exp_begin_cutoff_idx=500,
    exp_end_cutoff_idx=-500,
    seconds_per_subject_train=210,
    seconds_per_subject_test=210,    
    window_size=250,
    ocsvm_step_width=250,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_kernel="rbf",
    ocsvm_nu=None,
    ocsvm_gamma=None,
    feature_cols=[
        "x_a",
        "y_a",
        "z_a",
        "x_g",
        "y_g",
        "z_g",
    ],
    exclude_subjects=[],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM_125 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=125
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)


VALID_ROBUST_OCSVM_250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_2000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=2000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

# NORMALIZER_APPROACH (VALID)
# -----------------------------------------------------------
VALID_NORMALIZER_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-NORMALIZER-OCSVM",
    scaler="Normalizer",
    scaler_global=False,
    ocsvm_nu=0.074,
    ocsvm_gamma= 0.029,
)

In [5]:
P = VALID_ROBUST_OCSVM_2000
P.ocsvm_step_width = int(P.window_size * .5)
P.classifier="IF"

P.train_cores=1 # 20 cores for every user and 1 core for the nested crossval function
P.test_cores=2 # 10 cores for every user and 2 for the nested crossval function

# param_dist
param_dist = {
    "model__n_estimators": np.linspace(50, 200, num=10, dtype=np.int64),
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [10],
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [50, 100, 200],
    'model__max_samples':  np.arange(0.1, 1.01, 0.1),
    'model__contamination': np.linspace(0.0001, 0.1, 50),
    'model__max_features': np.arange(0.1, 1.01, 0.1),
    'model__warm_start': [True], 
    }

# param_dist = {
#     "model__n_estimators": [50, 100, 150, 200], #np.linspace(50, 200, num=10, dtype=np.int64),
#     'model__max_samples':  [.5,  .1],
#     'model__contamination': [0.01],
#     'model__max_features': [.5, 1.0],
#     'model__warm_start': [True], 
#     }

# param_dist = {
#     "model__n_estimators": [50, 100], #np.linspace(50, 200, num=10, dtype=np.int64),
#     'model__max_samples':  [.1],
#     'model__contamination': [0.01],
#     'model__max_features': [.5, 1.0],
#     'model__warm_start': [True], 
#     }

In [6]:
param_dist

{'model__n_estimators': [50, 100, 200],
 'model__max_samples': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__contamination': array([0.0001    , 0.00213878, 0.00417755, 0.00621633, 0.0082551 ,
        0.01029388, 0.01233265, 0.01437143, 0.0164102 , 0.01844898,
        0.02048776, 0.02252653, 0.02456531, 0.02660408, 0.02864286,
        0.03068163, 0.03272041, 0.03475918, 0.03679796, 0.03883673,
        0.04087551, 0.04291429, 0.04495306, 0.04699184, 0.04903061,
        0.05106939, 0.05310816, 0.05514694, 0.05718571, 0.05922449,
        0.06126327, 0.06330204, 0.06534082, 0.06737959, 0.06941837,
        0.07145714, 0.07349592, 0.07553469, 0.07757347, 0.07961224,
        0.08165102, 0.0836898 , 0.08572857, 0.08776735, 0.08980612,
        0.0918449 , 0.09388367, 0.09592245, 0.09796122, 0.1       ]),
 'model__max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__warm_start': [True]}

In [7]:
utils_ppp(P)

,Value
name,VALID-ROBUST-OCSVM
frequency,100
max_subjects,29
max_test_subjects,10
user_ids,"[1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 2..."
num_sample_points_per_exp,21000
exp_begin_cutoff_idx,500
exp_end_cutoff_idx,-500
seconds_per_subject_train,210
seconds_per_subject_test,210


In [8]:
utils_eer_scorer

make_scorer(utils_eer, greater_is_better=False)

In [9]:
#include 47 later
# user_ids = [9]
df_exps_dict = load_data_frames(P.user_ids, P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, P.num_sample_points_per_exp)
raw_dfList_exp1, raw_dfList_exp2 = df_exps_dict['dfList_exp1'], df_exps_dict['dfList_exp2']

Loading exp1 data:
1) accel_count: 28388, gyro_count: 31997
2) accel_count: 26010, gyro_count: 28954
3) accel_count: 28227, gyro_count: 31814
4) accel_count: 24860, gyro_count: 26105
5) accel_count: 24270, gyro_count: 24347
6) accel_count: 25012, gyro_count: 25060
7) accel_count: 25301, gyro_count: 25382
8) accel_count: 21975, gyro_count: 21658
19) accel_count: 24110, gyro_count: 25050
21) accel_count: 24326, gyro_count: 23809
22) accel_count: 29123, gyro_count: 28724
26) accel_count: 23148, gyro_count: 24291
27) accel_count: 24299, gyro_count: 23589
28) accel_count: 23807, gyro_count: 24523
29) accel_count: 24030, gyro_count: 23457
35) accel_count: 24388, gyro_count: 23673
36) accel_count: 24228, gyro_count: 24208
37) accel_count: 31945, gyro_count: 31816
38) accel_count: 22135, gyro_count: 22327
39) accel_count: 23573, gyro_count: 23459
40) accel_count: 23057, gyro_count: 24296
41) accel_count: 24102, gyro_count: 23681
42) accel_count: 24074, gyro_count: 24328
43) accel_count: 22631,

In [10]:
randomized_data_idx = list(range(len(P.user_ids)))
random.Random(SEED).shuffle(randomized_data_idx)
split_idx = 2 * (len(randomized_data_idx)//3) + 1
train_set = randomized_data_idx[: split_idx]
test_set = randomized_data_idx[split_idx: ]
# train_set = randomized_data_idx
print(f"train_set: {train_set}\ntest_set: {test_set}")
# train_set = test_set
# test_set = train_set
print(f"train_set: {train_set}\ntest_set: {test_set}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


In [11]:
num_sample_points_per_exp_user_47 = 18000
df_exps_dict_user_47 = load_data_frames([47], P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, num_sample_points_per_exp_user_47)
dfList_exp1_user_47, dfList_exp2_user_47 = df_exps_dict_user_47['dfList_exp1'], df_exps_dict_user_47['dfList_exp2']

raw_dfList_exp1_user_47 = dfList_exp1_user_47
raw_dfList_exp2_user_47 = dfList_exp2_user_47

Loading exp1 data:
47) accel_count: 22777, gyro_count: 22226
Loading exp2 data:
47) accel_count: 17718, gyro_count: 18353


In [ ]:
print(f"train_set: {train_set}")
# print(f"X_exp1_train_dic: {X_exp1_train_dic.keys()}")
# print(f"X_exp2_train_dic: {X_exp2_train_dic.keys()}")

In [ ]:
print(f"test_set: {test_set}")
# print(f"X_exp1_test_dic: {X_exp1_test_dic.keys()}")
# print(f"X_exp2_test_dic: {X_exp2_test_dic.keys()}")

# 1. Butterworth frequency Cut-off

## 1.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    




overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    
    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 41



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  14.4827835210599
MakeWACAXExpDicUnknown Time:  95.08045976795256
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13637.80it/s]


 33%|███▎      | 1/3 [06:33<13:06, 393.02s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14072.48it/s]


 67%|██████▋   | 2/3 [15:07<07:44, 464.62s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 10873.11it/s]


100%|██████████| 3/3 [23:10<00:00, 463.39s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18633.07it/s]


 20%|██        | 1/5 [00:11<00:47, 11.96s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 10057.08it/s]


 40%|████      | 2/5 [00:23<00:34, 11.48s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 17221.53it/s]


 60%|██████    | 3/5 [00:34<00:22, 11.37s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 10976.98it/s]


 80%|████████  | 4/5 [00:45<00:11, 11.45s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 10290.25it/s]


100%|██████████| 5/5 [00:57<00:00, 11.45s/it]

 22%|██▏       | 2/9 [58:56<3:20:42, 1720.40

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  4.4516099649481475
MakeWACAXExpDicUnknown Time:  25.269359302241355
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 19368.76it/s]


 33%|███▎      | 1/3 [07:11<14:22, 431.12s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15842.51it/s]


 67%|██████▋   | 2/3 [15:16<07:42, 462.79s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14413.42it/s]


100%|██████████| 3/3 [22:25<00:00, 448.39s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 15009.14it/s]


 20%|██        | 1/5 [00:09<00:39,  9.85s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12832.50it/s]


 40%|████      | 2/5 [00:19<00:28,  9.51s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 18067.22it/s]


 60%|██████    | 3/5 [00:28<00:18,  9.36s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 13543.12it/s]


 80%|████████  | 4/5 [00:37<00:09,  9.33s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 15709.00it/s]


100%|██████████| 5/5 [00:46<00:00,  9.36s/it]

 33%|███▎      | 3/9 [1:22:39<2:38:27, 1584.

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  3.5906891161575913
MakeWACAXExpDicUnknown Time:  18.114789451938123
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13171.00it/s]


## 1.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                                param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())



mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 49



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  12.72940863110125


# 2. Butterworth frequency Cut-off + EMA span
## 2.1 Naive Approach
### Optimizing and Testing

In [14]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]







overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")
        

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
total cut_off_span_pairs: (2401, 2), choice_num: 13


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 24, EMA span: 30



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  12.993648618925363
MakeWACAXExpDicUnknown Time:  69.987546721939
Done extracting features


...

window_size: 1750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 23
len_exp2_user_47: 19
MakeWACAXExpDicOwner Time:  2.2191082830540836
MakeWACAXExpDicUnknown Time:  6.8117343401536345
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 8005.16it/s]


 33%|███▎      | 1/3 [05:28<10:56, 328.19s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14210.75it/s]


 67%|██████▋   | 2/3 [11:04<05:32, 332.80s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13964.72it/s]


100%|██████████| 3/3 [16:46<00:00, 335.59s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18534.26it/s]


 20%|██        | 1/5 [00:06<00:24,  6.16s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14378.83it/s]


 40%|████      | 2/5 [00:12<00:18,  6.02s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 15158.31it/s]


 60%|██████    | 3/5 [00:17<00:11,  5.88s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 15969.18it/s]


 80%|████████  | 4/5 [00:23<00:05,  5.83s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 30897.27it/s]


100%|██████████| 5/5 [00:29<00:00,  5.86s/it]

 89%|████████▉ | 8/9 [2:32:41<17:57, 1077.40s

window_size: 2000
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 20
len_exp2_user_47: 17
MakeWACAXExpDicOwner Time:  2.0390960066579282
MakeWACAXExpDicUnknown Time:  6.189915192779154
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18192.60it/s]


 33%|███▎      | 1/3 [05:29<10:59, 329.69s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 21161.98it/s]


 67%|██████▋   | 2/3 [10:50<05:24, 324.28s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13101.06it/s]


100%|██████████| 3/3 [16:21<00:00, 327.18s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17761.19it/s]


 20%|██        | 1/5 [00:06<00:24,  6.07s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12808.99it/s]


 40%|████      | 2/5 [00:11<00:17,  5.96s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 15750.30it/s]


 60%|██████    | 3/5 [00:17<00:11,  5.82s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 22028.91it/s]


 80%|████████  | 4/5 [00:23<00:05,  5.81s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 8287.50it/s]


100%|██████████| 5/5 [00:29<00:00,  5.84s/it]

  0%|          | 0/3 [2:49:42<?, ?it/s]


NameError: name 'key_pair' is not defined

In [ ]:
1

## 2.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]






P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.EMA_per_win_span=P.span

    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 3. EMA span
## 3.1 Naive Approach
### Optimizing and Testing

In [18]:
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
rival_test_hyperparameters_df
print(rival_test_hyperparameters_file_name)
print(RIVAL_CLASSIFIER_TYPE_LST)

final_experiments_results/WACA-LOF/EMA_Mean_EER_Naive_df_test_dict_raw_df.json
['OCSVM', 'kNN', 'LOF']


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    
    EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

EMA span: 25



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.062254984863102
MakeWACAXExpDicUnknown Time:  70.15507365902886
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18575.31it/s]


 33%|███▎      | 1/3 [06:24<12:49, 384.78s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12274.81it/s]


 67%|██████▋   | 2/3 [12:31<06:14, 374.38s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13586.99it/s]


100%|██████████| 3/3 [19:05<00:00, 381.71s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18880.50it/s]


 20%|██        | 1/5 [00:06<00:27,  6.75s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 23373.11it/s]


 40%|████      | 2/5 [00:12<00:19,  6.35s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 11642.76it/s]


 60%|██████    | 3/5 [00:19<00:12,  6.38s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 14480.59it/s]


 80%|████████  | 4/5 [00:25<00:06,  6.27s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14011.37it/s]


100%|██████████| 5/5 [00:31<00:00,  6.30s/it]

 22%|██▏       | 2/9 [46:01<2:37:49, 1352.73

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  4.32451828988269
MakeWACAXExpDicUnknown Time:  19.267858140170574
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17795.10it/s]


 33%|███▎      | 1/3 [05:48<11:37, 348.95s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 16269.60it/s]


 67%|██████▋   | 2/3 [11:24<05:40, 340.90s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14055.98it/s]


100%|██████████| 3/3 [17:20<00:00, 346.80s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17288.97it/s]


 20%|██        | 1/5 [00:07<00:29,  7.44s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14844.47it/s]


 40%|████      | 2/5 [00:14<00:21,  7.22s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14657.71it/s]


 60%|██████    | 3/5 [00:21<00:14,  7.16s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 14883.97it/s]


 80%|████████  | 4/5 [00:28<00:07,  7.13s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14413.42it/s]


100%|██████████| 5/5 [00:35<00:00,  7.16s/it]

 33%|███▎      | 3/9 [1:04:22<2:03:45, 1237.

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  3.3790314039215446
MakeWACAXExpDicUnknown Time:  13.367395809851587
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 8494.79it/s]


 33%|███▎      | 1/3 [05:35<11:11, 335.68s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14354.22it/s]


 67%|██████▋   | 2/3 [10:59<05:28, 328.55s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13247.96it/s]


## 3.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    



overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    P.EMA_per_win_span=P.span

    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                    param_dist=param_dist) 
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

EMA span: 9



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.175995624158531
MakeWACAXExpDicUnknown Time:  87.51563867088407
Done extracting features


...


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  12.653837237041444
MakeWACAXExpDicUnknown Time:  89.56300481688231
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17147.60it/s]


 33%|███▎      | 1/3 [07:45<15:31, 465.96s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12580.40it/s]


 67%|██████▋   | 2/3 [15:09<07:32, 452.49s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 18157.16it/s]


100%|██████████| 3/3 [23:02<00:00, 460.82s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17509.10it/s]


 20%|██        | 1/5 [00:09<00:37,  9.34s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 20345.88it/s]


 40%|████      | 2/5 [00:18<00:27,  9.18s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 21361.36it/s]


 60%|██████    | 3/5 [00:27<00:18,  9.31s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 15488.57it/s]


 80%|████████  | 4/5 [00:37<00:09,  9.27s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 21307.11it/s]


100%|██████████| 5/5 [00:46<00:00,  9.24s/it]

 11%|█         | 1/9 [25:46<3:26:12, 1546.58

window_size: 250
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 167
len_exp2_user_47: 143
MakeWACAXExpDicOwner Time:  7.051525607705116
MakeWACAXExpDicUnknown Time:  44.71043225098401
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13595.80it/s]


 33%|███▎      | 1/3 [06:23<12:47, 383.78s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15139.16it/s]


# 4. SMA winsize
## 4.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 4.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")

    P.SMA_per_win_winsize=P.winsize


    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 5. Butterworth frequency Cut-off + SMA winsize
## 5.1 Naive Approach
### Optimizing and Testin

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]







overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 5.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

overlap=0.5
EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.SMA_per_win_winsize=P.winsize
    
    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")
    


    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 6. The effect of Varying Overlap

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")



time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for overlap in tqdm(OVERLAP_EXP_RANGE):
    overlap*=0.01
    max_window_size=2000
    step_width = int(max_window_size * (1-overlap))
    max_num_windows=max(len(getIndices(sampleSize=max_window_size, step=step_width, numSamplePoints=P.num_sample_points_per_exp)), 100)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    

    EER_df_train_dict[overlap] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                     extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                     param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\overlap: {overlap}\n")
        f.write(EER_df_train_dict[overlap].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=overlap, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["overlap"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# Summary

In [15]:

df=return_and_save_final_result_df_as_json(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)
df

ValueError: Unexpected character found when decoding 'false'

In [ ]:
relative_df=return_and_save_final_relative_result_df_as_json(df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, 
                                                             window_size_lst=WINDOW_SIZE_LST)
relative_df

In [ ]:
df.style.hide(axis='index').to_latex()


In [ ]:
relative_df.style.hide(axis='index').to_latex()
